In [1]:
# for correct relative imports
import sys; sys.path.append("../var_es_dgm")

In [2]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data import DataLoader, TensorDataset

from diffusers import DDPMScheduler
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pprint

from sklearn.preprocessing import StandardScaler

from var_es_dgm import TimeGrad
from var_es_dgm.basic_models import HistoricalSimulation, VarCov
from var_es_dgm.stat_tests import generate_report
from var_es_dgm.utils import seed_everything, compute_individual_returns, compute_portfolio_returns, estimate_var_es_torch

In [3]:
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_context("paper")

In [4]:
DATA_FOLDER = "../../data/"
df = pd.read_csv(DATA_FOLDER + "complete_stocks.csv")
df["Date"] = pd.to_datetime(df["Date"])

In [5]:
len(df["Ticker"].unique())

89

## 5%

In [6]:
res_timeGrad = list()
res_hist = list()
res_varcov = list()
alpha=0.05
RANDOM_STATE = 12

for i in range(5):
    # one complete cycle
    seed_everything(RANDOM_STATE + i)
    n_stocks = 10
    tickers = np.random.choice(df["Ticker"].unique(), n_stocks, replace=False)
    weights = 1/n_stocks
    print("Portfolio = {0}".format(" + ".join([f"{weights} * {i}"for i in tickers])))
    df_copy = df.loc[df["Ticker"].isin(tickers)].copy(deep=True)

    df_returns = compute_individual_returns(df_copy)
    df_returns = compute_portfolio_returns(df_returns)

    univariate_target = df_returns["Return_Target"]

    univariate_target = univariate_target.values[1:]
    train_size = df_returns[df_returns.Date <= "2022-06-01"].index[-1] + 1
    test_size = len(univariate_target) - train_size
    train = univariate_target[:train_size]

    ss = StandardScaler()
    train_scaled = torch.tensor(ss.fit_transform(train.reshape(-1, 1)), dtype=torch.float32)

    seed_everything(RANDOM_STATE)
    context_size = 90
    num_train_samples = 3000
    train_data = torch.zeros(num_train_samples, context_size, train_scaled.shape[1])
    train_target = torch.zeros(num_train_samples, 1, train_scaled.shape[1])
    train_idx = np.random.choice(np.arange(context_size, train_scaled.shape[0]), num_train_samples, replace=False)
    
    for i in tqdm(range(num_train_samples)):
        idx = train_idx[i]
        train_context = train_scaled[idx-context_size:idx]
        target_obs = train_scaled[idx]
        train_data[i] = train_context
        train_target[i] = target_obs

    # Create DataLoader for ease of torch training
    train_loader = DataLoader(TensorDataset(train_data, train_target), batch_size=128, shuffle=False)


    temp = torch.tensor(ss.transform(univariate_target.reshape(-1, 1)))
    test_data_context = torch.zeros(test_size, context_size, temp.shape[1])
    test_data_real = torch.zeros(test_size, 1, temp.shape[1])
    for i in range(test_size):
        idx = i + train_size
        test_data_context[i] = temp[idx-context_size:idx]
        test_data_real[i] = temp[idx]

    seed_everything(RANDOM_STATE)
    sheduler = DDPMScheduler(num_train_timesteps=30, beta_end=0.05, clip_sample=False)
    model = TimeGrad(1, 1, hidden_size=50, num_layers=2, scheduler=sheduler, num_inference_steps=30)
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    device = "cuda"
    n_epochs = 50
    model.to(device);

    model.fit(train_loader, optimizer, n_epochs, device)


    seed_everything(RANDOM_STATE)
    VaR_TimeGrad = torch.zeros(test_data_real.shape[0])
    ES_TimeGrad = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_TimeGrad[i], ES_TimeGrad[i] = estimate_var_es_torch(model, test, alpha=alpha, n_samples=500, device=device)

    res_timeGrad.append(generate_report(test_data_real.flatten(), VaR_TimeGrad, ES_TimeGrad, alpha=alpha))

    hist_sim = HistoricalSimulation(alpha=alpha)

    VaR_histSim = torch.zeros(test_data_real.shape[0])
    ES_histSim = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_histSim[i], ES_histSim[i] = hist_sim.predict(test)
    
    res_hist.append(generate_report(test_data_real.flatten(), VaR_histSim, ES_histSim, alpha=alpha))
    
    var_cov = VarCov(alpha=alpha)
    
    VaR_varCov = torch.zeros(test_data_real.shape[0])
    ES_varCov = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_varCov[i], ES_varCov[i] = var_cov.predict(test)
    
    res_varcov.append(generate_report(test_data_real.flatten(), VaR_varCov, ES_varCov, alpha=alpha))

    print(res_timeGrad[-1], res_hist[-1], res_varcov[-1])

Portfolio = 0.1 * INTC + 0.1 * DHR + 0.1 * COP + 0.1 * MRK + 0.1 * HON + 0.1 * T + 0.1 * MSFT + 0.1 * AXP + 0.1 * PEP + 0.1 * SYK


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

...

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

{'Kupicks POF': 0.2484438497515893, 'Haas TBF': 0.2830658575751374, 'Christoffersen Test': 0.5415791828627718, 'Kupicks Test': 0.2484438497515893, 'Acerbi Szekely 1': 1.9771578311920166, 'Acerbi Szekely 2': 0.0037446171045303345} {'Kupicks POF': 0.15902475231004004, 'Haas TBF': 0.7240770913383487, 'Christoffersen Test': 0.5415791828627718, 'Kupicks Test': 0.15902475231004004, 'Acerbi Szekely 1': 2.011117935180664, 'Acerbi Szekely 2': 0.003555006580427289} {'Kupicks POF': 0.02728075084397446, 'Haas TBF': 0.6011210401926721, 'Christoffersen Test': 0.5794958941032748, 'Kupicks Test': 0.02728075084397446, 'Acerbi Szekely 1': 1.9982151985168457, 'Acerbi Szekely 2': 0.0027752991300076246}


In [7]:
print(np.array([list(i.values()) for i in res_timeGrad]).mean(axis=0))
print(np.array([list(i.values()) for i in res_hist]).mean(axis=0))
print(np.array([list(i.values()) for i in res_varcov]).mean(axis=0))

[0.41440128 0.21366233 0.56657369 0.41440128 1.99850681 0.00359248]
[0.47226877 0.29528794 0.52562327 0.47226877 2.00618947 0.00414766]
[0.20164341 0.2664099  0.55322859 0.20164341 2.02576849 0.00348413]


## 1% 

In [8]:
res_timeGrad = list()
res_hist = list()
res_varcov = list()
alpha=0.01
RANDOM_STATE = 12

for i in range(5):
    # one complete cycle
    seed_everything(RANDOM_STATE + i)
    n_stocks = 10
    tickers = np.random.choice(df["Ticker"].unique(), n_stocks, replace=False)
    weights = 1/n_stocks
    print("Portfolio = {0}".format(" + ".join([f"{weights} * {i}"for i in tickers])))
    df_copy = df.loc[df["Ticker"].isin(tickers)].copy(deep=True)

    df_returns = compute_individual_returns(df_copy)
    df_returns = compute_portfolio_returns(df_returns)

    univariate_target = df_returns["Return_Target"]

    univariate_target = univariate_target.values[1:]
    train_size = df_returns[df_returns.Date <= "2022-06-01"].index[-1] + 1
    test_size = len(univariate_target) - train_size
    train = univariate_target[:train_size]

    ss = StandardScaler()
    train_scaled = torch.tensor(ss.fit_transform(train.reshape(-1, 1)), dtype=torch.float32)

    seed_everything(RANDOM_STATE)
    context_size = 90
    num_train_samples = 3000
    train_data = torch.zeros(num_train_samples, context_size, train_scaled.shape[1])
    train_target = torch.zeros(num_train_samples, 1, train_scaled.shape[1])
    train_idx = np.random.choice(np.arange(context_size, train_scaled.shape[0]), num_train_samples, replace=False)
    
    for i in tqdm(range(num_train_samples)):
        idx = train_idx[i]
        train_context = train_scaled[idx-context_size:idx]
        target_obs = train_scaled[idx]
        train_data[i] = train_context
        train_target[i] = target_obs

    # Create DataLoader for ease of torch training
    train_loader = DataLoader(TensorDataset(train_data, train_target), batch_size=128, shuffle=False)


    temp = torch.tensor(ss.transform(univariate_target.reshape(-1, 1)))
    test_data_context = torch.zeros(test_size, context_size, temp.shape[1])
    test_data_real = torch.zeros(test_size, 1, temp.shape[1])
    for i in range(test_size):
        idx = i + train_size
        test_data_context[i] = temp[idx-context_size:idx]
        test_data_real[i] = temp[idx]

    seed_everything(RANDOM_STATE)
    sheduler = DDPMScheduler(num_train_timesteps=30, beta_end=0.05, clip_sample=False)
    model = TimeGrad(1, 1, hidden_size=50, num_layers=2, scheduler=sheduler, num_inference_steps=30)
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    n_epochs = 50
    model.to(device);

    model.fit(train_loader, optimizer, n_epochs, device)


    seed_everything(RANDOM_STATE)
    VaR_TimeGrad = torch.zeros(test_data_real.shape[0])
    ES_TimeGrad = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_TimeGrad[i], ES_TimeGrad[i] = estimate_var_es_torch(model, test, alpha=alpha, n_samples=500, device=device)

    res_timeGrad.append(generate_report(test_data_real.flatten(), VaR_TimeGrad, ES_TimeGrad, alpha=alpha))

    hist_sim = HistoricalSimulation(alpha=alpha)

    VaR_histSim = torch.zeros(test_data_real.shape[0])
    ES_histSim = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_histSim[i], ES_histSim[i] = hist_sim.predict(test)
    
    res_hist.append(generate_report(test_data_real.flatten(), VaR_histSim, ES_histSim, alpha=alpha))
    
    var_cov = VarCov(alpha=alpha)
    
    VaR_varCov = torch.zeros(test_data_real.shape[0])
    ES_varCov = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_varCov[i], ES_varCov[i] = var_cov.predict(test)
    
    res_varcov.append(generate_report(test_data_real.flatten(), VaR_varCov, ES_varCov, alpha=alpha))

    print(res_timeGrad[-1], res_hist[-1], res_varcov[-1])

Portfolio = 0.1 * INTC + 0.1 * DHR + 0.1 * COP + 0.1 * MRK + 0.1 * HON + 0.1 * T + 0.1 * MSFT + 0.1 * AXP + 0.1 * PEP + 0.1 * SYK


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

...

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

{'Kupicks POF': 0.27384288964871895, 'Haas TBF': 0.6958701394864132, 'Christoffersen Test': 0.7862839203990928, 'Kupicks Test': 0.27384288964871895, 'Acerbi Szekely 1': 1.911399006843567, 'Acerbi Szekely 2': 9.653529559727758e-05} {'Kupicks POF': 0.9839089602677263, 'Haas TBF': 0.11777640619725117, 'Christoffersen Test': 0.7155219203146557, 'Kupicks Test': 0.9839089602677263, 'Acerbi Szekely 1': 1.9719154834747314, 'Acerbi Szekely 2': 0.00019918337056878954} {'Kupicks POF': 0.6124944699749673, 'Haas TBF': 0.35891287510350023, 'Christoffersen Test': 0.747017734035957, 'Kupicks Test': 0.6124944699749673, 'Acerbi Szekely 1': 1.91619074344635, 'Acerbi Szekely 2': 0.00014516596274916083}


In [9]:
print(np.hstack((np.nan_to_num(np.array([list(i.values()) for i in res_timeGrad])[:, :2], nan=0),
          np.nan_to_num(np.array([list(i.values()) for i in res_timeGrad])[:, 2:], nan=2))).mean(axis=0))

print(np.array([list(i.values()) for i in res_hist]).mean(axis=0))

print(np.hstack((np.nan_to_num(np.array([list(i.values()) for i in res_varcov])[:, :2], nan=0),
          np.nan_to_num(np.array([list(i.values()) for i in res_varcov])[:, 2:], nan=2))).mean(axis=0))

[4.22455322e-01 4.65255942e-01 7.34845583e-01 4.22455322e-01
 1.92427626e+00 1.75796497e-04]
[4.74905851e-01 3.58427983e-01 7.11934199e-01 4.74905851e-01
 2.05907185e+00 2.26448207e-04]
[6.53381145e-01 4.03996225e-01 7.53200902e-01 6.53381145e-01
 2.07649233e+00 1.56829669e-04]
